# Using Metadata to Improve Artifical Intelligance Medical Image Diagnostic Accuracy
**Purpose and Background**
Conduct a descriptive analysis of crowdsourced data extracted from user interaction with a mobile application where tasked to binarly (yes or no) identify abnormalities in medical images. 

Two user categories were differentiated: Medical experts hired to interact with the application; and crowd, anyone who downloaded and used the application.

**Show that the crowd agrees with the expert majority more than experts agreeing with the expert majority**


### Import datasets

In [1]:
import pandas as pd
import numpy as np
results = pd.read_csv('1345_customer_results.csv') #medical case results
admin = pd.read_csv('1345_admin_reads.csv') #raw individual read

### Inspect Customer Results

In [2]:
results.dtypes
results = results.set_index('Case ID')

**Preliminary filtering for security purposes**


In [3]:
results = results.dropna(subset=['Origin']) 
results["Expert: Abnormal Votes"] = results["Origin"].str.extract(r'vote(\d)').astype(float)
results = results.drop(['Origin Created At','Origin','Content ID','URL'],axis=1)

In [4]:
results.head(2)

,Labeling State,Series,Series Index,Patch,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Internal Notes,Comments,Explanation,Expert: Abnormal Votes
Case ID,,,,,,,,,,,,,,,,,,,
5888087,Gold Standard,NaN,NaN,NaN,2,'no','no',0.0,1.0,'no',2,1.54,'yes',0,0.0,NaN,[],NaN,2.0
5888088,Gold Standard,NaN,NaN,NaN,3,'no','no',0.0,1.0,'no',3,2.34,'yes',0,0.0,NaN,[],NaN,0.0


Any rows that did not have a string associated with expert votes in the URL were dropped (i.e. NA)

*0 means all experts thought the case was normal  

In [5]:
results = results.dropna(subset=["Expert: Abnormal Votes"])
#results["Expert: Abnormal Votes"].isnull().any()

**Inspect NaN Columns for Content**

In [6]:
results.loc[results['Series'].notna()| results['Series Index'].notna() | results['Patch'].notna() | results['Internal Notes'].notna() | results['Explanation'].notna()]

,Labeling State,Series,Series Index,Patch,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Internal Notes,Comments,Explanation,Expert: Abnormal Votes
Case ID,,,,,,,,,,,,,,,,,,,


Dataframe is empty; None of the columns scanned through the pipeline contained any data

In [7]:
results = results.drop(['Series','Series Index','Patch','Internal Notes','Explanation'],axis=1)

**Inspect Comments for Relevance**

In [8]:
results[results['Comments'] != '[]']


,Labeling State,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Comments,Expert: Abnormal Votes
Case ID,,,,,,,,,,,,,,
5892332,Gold Standard,1,'no','no',0.000,1.000,'no',1,0.80,'yes',0,0.00,['There was rapid and spiky rates so why am I ...,3.0
5894116,Gold Standard,5,'no','yes',1.000,1.000,'yes',5,4.00,'no',0,0.00,['Can someone explain why the answer is “no”?'],0.0
5896433,Gold Standard,3,'yes','no',1.000,1.000,'no',3,2.32,'yes',0,0.00,['??'],5.0
5899520,Gold Standard,2,'yes','no',1.000,1.000,'no',2,1.58,'yes',0,0.00,"[""i can't see any spike in this question so wh...",5.0
5900998,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.56,'no',0,0.00,['There is obviously a peak happened in there'],3.0
5901914,Gold Standard,6,'yes','no',1.000,1.000,'no',6,4.72,'yes',0,0.00,['No spike present'],5.0
5902040,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.58,'no',0,0.00,['How?'],3.0
5904120,Gold Standard,1,'yes','no',1.000,1.000,'no',1,0.78,'yes',0,0.00,['How? '],6.0
5904413,Gold Standard,3,'yes','no',1.000,1.000,'no',3,2.46,'yes',0,0.00,['Multiple?'],6.0


None of the comments seem relevant, so the Comments column will be dropped

In [9]:
results = results.drop(['Comments'],axis=1)

There should only be 8 experts total; cases were dropped for experts count >8

In [10]:
results = results[results["Expert: Abnormal Votes"] <= 8]

### Important columns for analysis; original metadata
Each row corresponds to a medical case 

**Identifiers:** 

Case ID: unique identifier will serve as index

Labeling State: identifies whether a expert consensus has been achieved (yes=Gold Standard, no= In Progress)

URL: Extracted out expert vote count within the URL 

**Reads and Annotations**

Qualified Reads: total crowd vote count

Expert: Abnormal Votes: number of experts who thought the case was abnormal

(note, the total of experts voting is always 8)

Correct Label: overall expert consensus 

{yes=case is abnormal, no=case is normal, NaN=no consensus}

Majority Label: overall crowd consensus on each case

**Measures of Confidence**

Difficulty: Qualified Reads *without the Correct Label* divided by total Qualified Reads.

Agreement: Qualified Reads *with the Majority Label* divided by total Qualified Reads.

Nth Choice Answer: crowd answer (First Choice is the Majority Label)
        
Nth Choice Votes: number of crowd votes per answer
        
Nth Choice Weight:
        
        
        



### Add Additional Relevant Columns 




In [55]:
data = results
data.dropna(inplace = True)
perc = [.2,.4,.6,.8]
desc = data['Difficulty'].describe(percentiles = perc)
print(desc)

count    23758.000000
mean         0.244252
std          0.334227
min          0.000000
20%          0.000000
40%          0.000000
50%          0.000000
60%          0.200000
80%          0.500000
max          1.000000
Name: Difficulty, dtype: float64


cluster cases categorically based on difficulty; can calcualte means and other statistics from there

In [56]:
df = results 
expert_count = 8

#first digit

FirstDigit = {
'0' : 'very easy',
'1' : 'very challenging'
}

#second digit
SecondDigit = {
    '1' : 'very easy',
    '2' : 'easy',
    '3' : 'easy',
    '4' : 'moderate',
    '5' : 'moderate',
    '6' : 'moderately challenging',
    '7' : 'moderately challenging',
    '8' : 'challenging',
    '9' : 'challenging'
}


#df['Difficulty Category'] = df['Difficulty'].apply(lambda num: SecondDigit[str(num)[3]])
#df['Difficulty Category'] = df['Difficulty'].apply(lambda num: FirstDigit[str(num)[1]])


In [11]:
df = results 
expert_count = 8
df["Expert: Normal Votes"] = (expert_count - results["Expert: Abnormal Votes"])
df["Expert Agreement"] = df["Expert: Abnormal Votes"]/expert_count
#split_conses = (df.loc(df["Expert/Expert Agreement"]==0.5))
df

,Labeling State,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Expert: Abnormal Votes,Expert: Normal Votes,Expert Agreement
Case ID,,,,,,,,,,,,,,,
5888087,Gold Standard,2,'no','no',0.000,1.000,'no',2,1.54,'yes',0,0.00,2.0,6.0,0.250
5888088,Gold Standard,3,'no','no',0.000,1.000,'no',3,2.34,'yes',0,0.00,0.0,8.0,0.000
5888089,Gold Standard,2,'no','no',0.000,1.000,'no',2,1.70,'yes',0,0.00,0.0,8.0,0.000
5888090,Gold Standard,1,'no','no',0.000,1.000,'no',1,0.82,'yes',0,0.00,0.0,8.0,0.000
5888091,In Progress,7,NaN,'yes',NaN,0.571,'yes',4,3.28,'no',3,2.32,4.0,4.0,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918375,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.56,'no',0,0.00,2.0,6.0,0.250
5918376,Gold Standard,3,'no','yes',0.667,0.667,'yes',2,1.56,'no',1,0.76,3.0,5.0,0.375
5918377,In Progress,6,NaN,'yes',NaN,1.000,'yes',6,4.78,'no',0,0.00,4.0,4.0,0.500


For each case:
Expert: Normal Votes: I subtracted the number of total experts by the known number of experts who voted the case as abnormal
Expert Agreement: I divided the number of experts who voted the case as abnormal by the total number of experts to get the porportion of experts who agree that the case is abnormal.
Error Rate: I extracted the indexes for each category and calculated the "error rate" for the experts who did not vote for the expert majority

#### I will rename some of the original columns for clarity

    {Original column --> Renamed Column}
    
    Correct Label --> Expert Majority

    Majority Label --> Crowd Majority

    Difficulty --> Expert/Crowd Disagreement

    Agreement --> Crowd Agreement

In [12]:
df["Expert Majority"] = results["Correct Label"]
df["Crowd Majority"] = results["Majority Label"]
df["Expert/Crowd Disagreement"] = results["Difficulty"] #porportion of crowd disagreeing with expert consensus
df["Crowd Agreement"] = results["Agreement"] #porportion of crowd agreeing with crowd consensus
df = df.drop(columns= ["Correct Label","Majority Label","Difficulty","Agreement"])


df['Consensus'] = np.where(df['Expert Majority'] == df['Crowd Majority'],'yes','no')

#### Error rate of experts
I extracted the indexes for each category and calculated the "error rate" for the experts who did not vote for the expert majority

In [13]:
EM_yes = df.index[df['Expert Majority'] == "'yes'"].tolist()

EM_no = df.index[df['Expert Majority'] == "'no'"].tolist()

df.loc[EM_yes,"Error Rate"]= df['Expert: Normal Votes'][EM_yes]/expert_count
df.loc[EM_no,"Error Rate"]= df['Expert: Abnormal Votes'][EM_no]/expert_count
df.fillna('', inplace=True)
beg_index = list(df.columns).index('Expert: Abnormal Votes') #9
df.iloc[ : , 13:]


,Expert/Crowd Disagreement,Crowd Agreement,Consensus,Error Rate
Case ID,,,,
5888087,0.0,1.0,yes,0.25
5888088,0.0,1.0,yes,0.0
5888089,0.0,1.0,yes,0.0
5888090,0.0,1.0,yes,0.0
5888091,,0.571,no,
...,...,...,...,...
5918375,1.0,1.0,no,0.25
5918376,0.667,0.667,no,0.375
5918377,,1.0,no,


## Exploratory Analysis 1

In [14]:
#%pip install jupyter-dash
import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as ff
pio.renderers.default='notebook'
import matplotlib.pyplot as plt
#ax = plt.subplot()

In [15]:
#FIX NaN; currently is blank
df.fillna('', inplace=True)
print(df['Expert Majority'].value_counts())

'no'     12000
'yes'    12000
          3000
Name: Expert Majority, dtype: int64


12,000 medical cases were judged to be abnormal by experts

12,000 medical cases were judged to be normal by experts

3,0000 medical cases failed to reach a consensus (4 experts voted for normal and 4 experts voted for abnormal)


### Exploratory Analysis

In [53]:
fig = px.histogram(df,x= "Qualified Reads", color="Error Rate")
fig.update_layout(xaxis_range=[0,15])
fig.show()

In [ ]:
print(len(df[df['Qualified Reads'] < 5].value_counts()))


3639


#### How reliable are the individual experts on average?

In [ ]:

fig = px.density_heatmap(df, x="Expert Majority", y='Crowd Majority',text_auto=True)
fig.show()
df.std(axis=0)


/var/folders/qt/mmxjlbbn0lzdwmjghjtjwlc00000gn/T/ipykernel_31230/3685616616.py:3: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



Qualified Reads           2.549289
First Choice Votes        2.360711
First Choice Weight       1.888224
Second Choice Votes       0.959345
Second Choice Weight      0.764048
Expert: Abnormal Votes    2.582037
Expert: Normal Votes      2.582037
Expert Agreement          0.322755
dtype: float64

When the experts are undecided (N=4) on the case prognosis, crowd appears to have a more unified opinion on the case. Let's make a histogram examining the cases where there's lack of consensus.

With the number of experts being 8, filtering the qualified reads to 5 or more would keep things more porportional

In [19]:
filt_df = df[df['Qualified Reads'] >= 5]
fig = px.density_heatmap(filt_df, x="Expert Majority", y='Crowd Majority',text_auto=True)
fig.show()
filt_df.std(axis=0)

/var/folders/qt/mmxjlbbn0lzdwmjghjtjwlc00000gn/T/ipykernel_31230/701681104.py:4: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



Qualified Reads           1.942279
First Choice Votes        1.961178
First Choice Weight       1.569474
Second Choice Votes       1.137077
Second Choice Weight      0.905635
Expert: Abnormal Votes    2.456104
Expert: Normal Votes      2.456104
Expert Agreement          0.307013
Crowd Agreement           0.153559
dtype: float64

In [20]:
fig = px.scatter(filt_df,x= "Expert Agreement", y="Crowd Agreement")
#fig.update_layout(yaxis_range=[0.4,1.1])
fig.show()

This doesn't tell us much; perhaps calculate an average?

In [21]:
consensus_no = filt_df.index[filt_df['Consensus']=='no'].tolist()
fig = px.histogram(filt_df.loc[consensus_no], x='Crowd Agreement',color='Crowd Majority',marginal='box')
fig.update_layout(xaxis_range=[-0.1,1.1])
fig.show()

Move red to left side of the graph, overlay this graph onto graph below

In [22]:
fig = px.histogram(filt_df.loc[consensus_no], x='Expert Agreement',color='Expert Majority',marginal='box')
fig.show()

### Let's check out what are the trends when experts are split in their opinion:


In [23]:
experts_split = filt_df.index[filt_df['Expert Agreement']==0.500].tolist()
fig = px.histogram(filt_df.loc[experts_split], x='Crowd Agreement',color='Crowd Majority',marginal='box')
fig.update_layout(xaxis_range=[0.45,1.05])
fig.show()

## Trying to aggregate case ids

In [24]:

filt_df = df[df['Qualified Reads'] > 5]
#fig = px.histogram(filt_df,x= "Case ID")
#fig.update_layout(yaxis_range=[0.4,1.1])
#fig.show()

per case ID:
sum the instances and show porportions of error rate; ### aggregate case IDs!!

In [25]:
#import statistics
#filt_df['aggreg_error_rate'] = filt_df.groupby(['Case ID'], as_index = False)['Error Rate'].mean()
#filt_df['aggreg_error_rate'] = filt_df.groupby(['Case ID'], as_index = False).apply(lambda x: statistics.mean(x))
#def average(x):
   # x/sum(x)
#print(average(df['Expert: Normal Votes']))
#caseid_agg = df.groupby(['Case ID'])['Expert Agreement'].aggregate{}
#print(df['Case ID'].unique)
#fig = px.histogram(caseid_agg)
#fig.show()
#print(sum(caseid_agg))
#print(df.index[-1] - df.index[1])


In [27]:

#filt_df = df[df['Qualified Reads'] > 5]
#fig = px.histogram(filt_df,x="Case ID", color="Error Rate")
#fig.update_layout(yaxis_range=[0.45,1.1])
#fig.show()

Seems uniform; which cases fall into the category? Are they the harder ones?

Make these into bubble charts to show porportion: https://plotly.com/python/bubble-charts/

### Problem x User Matrix

#### Isolate Problem_id, User_id, accuracy, chosen answer, and correct answer

In [28]:
admin = pd.read_csv('1345_admin_reads.csv') #raw individual read

In [29]:
admin
#results = results.set_index('Case ID')


,topic_id,problem_id,user_id,read_id,labeling_state,patch,score,accuracy,contest_id,mission_id,content_id,chosen_answer,origin,origin_created_at,series,series_index,answerChoiceIds,response_submitted_at,problem_appeared_at
0,1345,5888087,55058,132610888,gold_standard,NaN,100,0.78,8011,NaN,3264386,['no'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 21:41:47.756010+00:00,NaN,NaN,[25796611],2022-04-12 09:20:24.912000+00:00,2022-04-12 09:20:24.415999+00:00
1,1345,5888087,248277,107696869,gold_standard,NaN,100,0.76,6437,NaN,3264386,['no'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 21:41:47.756010+00:00,NaN,NaN,[25796611],2021-12-04 15:00:58.596999+00:00,2021-12-04 15:00:56.760999+00:00
2,1345,5888088,19769,60673858,gold_standard,NaN,100,0.78,5183,NaN,3264387,['no'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 21:41:47.824498+00:00,NaN,NaN,[25796612],2021-08-09 07:41:17.754000+00:00,2021-08-09 07:41:16.961000+00:00
3,1345,5888088,237039,99676200,gold_standard,NaN,100,0.80,6151,NaN,3264387,['no'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 21:41:47.824498+00:00,NaN,NaN,[25796612],2021-11-01 12:05:16.948999+00:00,2021-11-01 12:05:16.076000+00:00
4,1345,5888088,280445,141519169,gold_standard,NaN,100,0.76,9076,NaN,3264387,['no'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 21:41:47.824498+00:00,NaN,NaN,[25796612],2022-06-10 12:16:45.950000+00:00,2022-06-10 12:16:45.112000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152055,1345,5918379,19769,60689328,gold_standard,NaN,100,0.82,5183,NaN,3294678,['yes'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 22:00:15.397399+00:00,NaN,NaN,[25796257],2021-08-09 10:17:10.168000+00:00,2021-08-09 10:17:09.165000+00:00
152056,1345,5918379,53129,61096537,gold_standard,NaN,100,0.78,5183,NaN,3294678,['yes'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 22:00:15.397399+00:00,NaN,NaN,[25796257],2021-08-11 18:34:25.122999+00:00,2021-08-11 18:34:23.250999+00:00
152057,1345,5918379,102777,99217212,gold_standard,NaN,100,0.84,6151,NaN,3294678,['yes'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 22:00:15.397399+00:00,NaN,NaN,[25796257],2021-10-31 00:28:19.111000+00:00,2021-10-31 00:28:17.107000+00:00
152058,1345,5918379,137347,114399651,gold_standard,NaN,100,0.76,6810,NaN,3294678,['yes'],https://centaur-customer-uploads.s3.us-east-1....,2021-07-26 22:00:15.397399+00:00,NaN,NaN,[25796257],2022-01-07 16:10:33.714999+00:00,2022-01-07 16:10:32.913000+00:00


In [30]:
PU_df = admin
PU_df = PU_df.set_index('read_id')
PU_df = admin[['problem_id','user_id','accuracy','chosen_answer']].copy()
PU_df = pd.concat([filt_df[['Expert Majority']],PU_df],axis=1)
PU_df

,Expert Majority,problem_id,user_id,accuracy,chosen_answer
5888091,,NaN,NaN,NaN,NaN
5888093,'no',NaN,NaN,NaN,NaN
5888100,'no',NaN,NaN,NaN,NaN
5888102,'no',NaN,NaN,NaN,NaN
5888104,'no',NaN,NaN,NaN,NaN
...,...,...,...,...,...
152055,NaN,5918379.0,19769.0,0.82,['yes']
152056,NaN,5918379.0,53129.0,0.78,['yes']
152057,NaN,5918379.0,102777.0,0.84,['yes']
152058,NaN,5918379.0,137347.0,0.76,['yes']


In [32]:
PU_matrix_accur = PU_df.pivot_table(index='problem_id',columns='user_id',values='accuracy', aggfunc='mean')
PU_matrix_accur

user_id,1196.0,1858.0,2581.0,2815.0,2861.0,7280.0,11697.0,11715.0,15245.0,19769.0,...,292079.0,292198.0,292506.0,292546.0,292650.0,292669.0,292741.0,292812.0,292855.0,293188.0
problem_id,,,,,,,,,,,,,,,,,,,,,
5888087.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5888088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5888089.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5888090.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5888091.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918374.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5918375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5918376.0,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
user_accuracy_table = pd.DataFrame(PU_matrix_accur.mean())
user_accuracy_table

,0
user_id,
1196.0,0.809749
1858.0,0.770667
2581.0,0.769310
2815.0,0.750000
2861.0,0.835833
...,...
292669.0,0.785000
292741.0,0.772400
292812.0,0.768696


Average accuracy of each user

In [52]:
#PU_matrix_corr = PU_df.pivot_table(index='problem_id',columns='user_id',values='Expert Majority', aggfunc='sum')
#PU_matrix_corr
PU_matrix_accur.keys()

Float64Index([  1196.0,   1858.0,   2581.0,   2815.0,   2861.0,   7280.0,
               11697.0,  11715.0,  15245.0,  19769.0,
              ...
              292079.0, 292198.0, 292506.0, 292546.0, 292650.0, 292669.0,
              292741.0, 292812.0, 292855.0, 293188.0],
             dtype='float64', name='user_id', length=1034)

### SciKit/Weighted Statistics

In [ ]:
pip install deslib

META-DES 7, 8, 15
: R. M. O. Cruz, R. Sabourin, G. D. C. Cavalcanti, T. I. Ren, META-DES: A dynamic ensemble selection framework using meta-learning, Pattern Recognition 48 (5) (2015) 1925–1935.

Cruz, R.M., Sabourin, R. and Cavalcanti, G.D., 2015, July. META-DES. H: a dynamic ensemble selection technique using meta-learning and a dynamic weighting approach. In Neural Networks (IJCNN), 2015 International Joint Conference on (pp. 1-8)

R. M. O. Cruz, R. Sabourin, G. D. C. Cavalcanti, META-DES.Oracle: Meta-learning and feature selection for dynamic ensemble selection, Information Fusion 38 (2017) 84–103.Nov 30;38:84-103.


 This method (Dynamic Ensemble Selection-Kullback-Leibler divergence (DES-KL).) estimates the competence of the classifier (the ability of the user to answer correctly) and puts them into clusters categorizing groups that answer more accurately than others.

https://github.com/scikit-learn-contrib/DESlib/blob/master/deslib/des/probabilistic/deskl.py


In [51]:
import numpy as np

from deslib.des.probabilistic import BaseProbabilistic
from deslib.util import entropy_func


class DESKL(BaseProbabilistic):
    
    def __init__(self, pool_classifiers=None, k=None, DFP=False, with_IH=False,
                 safe_k=None, IH_rate=0.30, mode='selection',
                 random_state=None, knn_classifier='knn',
                 knn_metric='minkowski', DSEL_perc=0.5, n_jobs=-1,
                 voting='hard'):
        super(DESKL, self).__init__(pool_classifiers=pool_classifiers,
                                    k=k,
                                    DFP=DFP,
                                    with_IH=with_IH,
                                    safe_k=safe_k,
                                    IH_rate=IH_rate,
                                    mode=mode,
                                    random_state=random_state,
                                    knn_classifier=knn_classifier,
                                    knn_metric=knn_metric,
                                    DSEL_perc=DSEL_perc,
                                    n_jobs=n_jobs,
                                    voting=voting)

        self.selection_threshold = 0.0

    def source_competence(self):
        """Calculates the source of competence using the KL divergence method.

        The source of competence C_src at the validation point
        :math:`\\mathbf{x}_{k}` is calculated by the KL divergence
        between the vector of class supports produced by the base classifier
        and the outputs of a random classifier (RC) RC = 1/L, L being the
        number of classes in the problem. The value of C_src is negative if
        the base classifier misclassified the instance :math:`\\mathbf{x}_{k}`.

        Returns
        ----------
        C_src : array of shape (n_samples, n_classifiers)
            The competence source for each base classifier at each data point.
        """

        C_src = np.zeros((self.n_samples_, self.n_classifiers_))
        for clf_index in range(self.n_classifiers_):
            supports = self.dsel_scores_[:, clf_index, :]
            is_correct = self.DSEL_processed_[:, clf_index]
            C_src[:, clf_index] = entropy_func(self.n_classes_, supports,
                                               is_correct)

        return C_src
    
DESKL(PU_matrix_accur.keys(),k=10, knn_metric='minkowski')

TypeError: BaseProbabilistic.__init__() got an unexpected keyword argument 'knn_metric'